In [1]:
%load_ext tensorboard
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime
import os
import scipy.io
import tarfile
from PIL import Image
from tqdm.notebook import tqdm_notebook
import sklearn
import tensorflow as tf
from sklearn.model_selection import train_test_split
print(tf.__version__)

2.0.0


In [2]:
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)

True

In [3]:
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz

--2019-12-16 18:54:17--  http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344862509 (329M) [application/x-gzip]
Saving to: ‘102flowers.tgz’

102flowers.tgz      100%[===================>] 328.89M  86.4MB/s    in 4.0s    

2019-12-16 18:54:21 (83.0 MB/s) - ‘102flowers.tgz’ saved [344862509/344862509]



In [4]:
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat

--2019-12-16 18:54:21--  http://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 502
Saving to: ‘imagelabels.mat’

imagelabels.mat     100%[===================>]     502  --.-KB/s    in 0s      

2019-12-16 18:54:21 (99.3 MB/s) - ‘imagelabels.mat’ saved [502/502]



In [5]:
labels = scipy.io.loadmat('imagelabels.mat')
labels = np.array(labels['labels'][0])

In [6]:
imageFile  = tarfile.open(name='102flowers.tgz', mode='r:gz')

savepath = '/tmp/102flowers/'

imageFile.extractall(savepath)

In [7]:
train_path = '/tmp/train/'
test_path = '/tmp/test/'
if not os.path.isdir(train_path):
  os.mkdir(train_path)
if not os.path.isdir(test_path):
  os.mkdir(test_path)

In [8]:
count_M20 = 0
for i in range(101):
    rand = np.random.randint(0, 100, dtype= np.int64)
    if rand > 20:
        count_M20 += 1
print(count_M20, '%')

81 %


In [9]:
path = savepath + 'jpg/'
np.random.seed(42)
for f in tqdm_notebook(os.listdir(path), total= len(os.listdir(path))):
  image = Image.open(os.path.join(path, f))
  image_id = int(f.replace('image_', "").replace('.jpg',""))
  rand = np.random.randint(0,100,dtype= np.int64)
  if rand > 20:
    if not os.path.isdir(train_path + str(labels[image_id-1])):
        os.mkdir(train_path + str(labels[image_id-1]))
    image.save(train_path + str(labels[image_id-1]) + '/' + f )
  else:
    if not os.path.isdir(test_path + str(labels[image_id-1])):
        os.mkdir(test_path + str(labels[image_id-1]))
    image.save(test_path + str(labels[image_id-1]) + '/' + f )

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class FixedImageDataGenerator(ImageDataGenerator):
    def standardize(self, x):
        if self.featurewise_center:
            x = ((x/255.) - 0.5) * 2.
        return x
    
data_gen_args = dict(featurewise_center=True,
                     featurewise_std_normalization=True,
                     rotation_range=90,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2,
                     rescale = 1/255.,
                     validation_split=0.22)

#train_datagen = ImageDataGenerator(rescale = 1/255., validation_split=0.2)

train_datagen = FixedImageDataGenerator(**data_gen_args)

train_generator = train_datagen.flow_from_directory(train_path, subset='training', target_size = (500,500), batch_size = 16, class_mode = 'categorical', seed=42, shuffle=True)

valid_generator = train_datagen.flow_from_directory(train_path, subset='validation', target_size = (500,500), batch_size = 16, class_mode = 'categorical', seed=42, shuffle=True)

#train_generator = train_datagen.flow(imgs, y_one_hot, batch_size=128, subset='training', seed=42)

test_datagen = FixedImageDataGenerator(rescale= 1/255.)

test_generator = test_datagen.flow_from_directory(test_path, target_size= (500,500), batch_size = 16, class_mode = 'categorical', seed = 42, shuffle=True)

Found 5071 images belonging to 102 classes.
Found 1361 images belonging to 102 classes.
Found 1757 images belonging to 102 classes.


In [11]:
IMG_WIDTH, IMG_HEIGHT, CHANNELS = (500,500,3)
eps = 1.1e-5
concat_axis = 3

In [12]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Layer, InputSpec
try:
    from tensorflow.keras import initializations
except ImportError:
    from tensorflow.keras import initializers as initializations

class Scale(Layer):
    '''Custom Layer for DenseNet used for BatchNormalization.
    
    Learns a set of weights and biases used for scaling the input data.
    the output consists simply in an element-wise multiplication of the input
    and a sum of a set of constants:
        out = in * gamma + beta,
    where 'gamma' and 'beta' are the weights and biases learnt.
    # Arguments
        axis: integer, axis along which to normalize in mode 0. For instance,
            if your input tensor has shape (samples, channels, rows, cols),
            set axis to 1 to normalize per feature map (channels axis).
        momentum: momentum in the computation of the
            exponential average of the mean and standard deviation
            of the data, for feature-wise normalization.
        weights: Initialization weights.
            List of 2 Numpy arrays, with shapes:
            `[(input_shape,), (input_shape,)]`
        beta_init: name of initialization function for shift parameter
            (see [initializations](../initializations.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
        gamma_init: name of initialization function for scale parameter (see
            [initializations](../initializations.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
    '''
    def __init__(self, weights=None, axis=-1, momentum = 0.9, beta_init='zero', gamma_init='one', **kwargs):
        self.momentum = momentum
        self.axis = axis
        self.beta_init = initializations.get(beta_init)
        self.gamma_init = initializations.get(gamma_init)
        self.initial_weights = weights
        super(Scale, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        shape = (int(input_shape[self.axis]),)

        self.gamma = tf.Variable(self.gamma_init(shape), name='{}_gamma'.format(self.name))
        self.beta = tf.Variable(self.beta_init(shape), name='{}_beta'.format(self.name))
        #self.trainable_weights = [self.gamma, self.beta]

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights

    def call(self, x, mask=None):
        input_shape = self.input_spec[0].shape
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis]

        out = tf.reshape(self.gamma, broadcast_shape) * x + tf.reshape(self.beta, broadcast_shape)
        return out

    def get_config(self):
        config = {"momentum": self.momentum, "axis": self.axis}
        base_config = super(Scale, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

def conv_block(x, stage, branch, nb_filter, dropout_rate = None, weight_decay = 1e-4):
  conv_name_base = 'conv' + str(stage) + '_' + str(branch)
  relu_name_base = 'relu' + str(stage) + '_' + str(branch)
  #1X1 convolution
  inter_channel = nb_filter * 4
  x = BatchNormalization(epsilon=eps, axis=concat_axis, name = conv_name_base + '_x1_bn')(x)
  x = Scale(axis=concat_axis, name=conv_name_base + '_x1_scale')(x)
  x = Activation('relu', name= relu_name_base + '_x1')(x)
  x = Conv2D(inter_channel, (1,1), name=conv_name_base + '_x1', use_bias=False)(x)
  
  if dropout_rate:
    x = Dropout(dropout_rate)(x)

  #3X3 convolution
  x = BatchNormalization(epsilon=eps, axis= concat_axis, name=conv_name_base + '_x2_bn')(x)
  x = Scale(axis= concat_axis, name = conv_name_base + '_x2_scale')(x)
  x = Activation('relu', name= relu_name_base + '_x2')(x)
  x = ZeroPadding2D((1,1), name= conv_name_base + '_x2_zeropadding')(x)
  x = Conv2D(nb_filter, (3,3), name=conv_name_base + '_x2', use_bias=False)(x)

  if dropout_rate:
    x = Dropout(dropout_rate)(x)
  
  return x

def transition_block(x, stage, nb_filter, compression= 1.0, dropout_rate = None, weight_decay = 1e-4):
  conv_name_base = 'conv' + str(stage) + '_blk'
  relu_name_base = 'relu' + str(stage) + '_blk'
  pool_name_base = 'pool' + str(stage)
  x = BatchNormalization(epsilon=eps, axis = concat_axis, name=conv_name_base + '_bn')(x)
  x = Scale(axis=concat_axis, name=conv_name_base+'_scale')(x)
  x = Activation('relu', name= relu_name_base)(x)
  x = Conv2D(int(nb_filter * compression), (1,1), name= conv_name_base, use_bias=False)(x)
  if dropout_rate:
    x = Dropout(dropout_rate)(x)
  x = AveragePooling2D((2,2), strides=(2,2), name = pool_name_base)(x)
  return x

def dense_block(x, stage, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay= 1e-4, grow_nb_filters= True):
  concat_feat = x
  for i in range(nb_layers):
    branch = i + 1
    x = conv_block(concat_feat, stage, branch, growth_rate, dropout_rate, weight_decay)
    concat_feat = concatenate([concat_feat, x], axis=concat_axis, name = 'concat_'+str(stage)+'_'+str(branch))

    if grow_nb_filters:
      nb_filter += growth_rate
  return concat_feat, nb_filter

def DenseNet(nb_dense_block=4, growth_rate=32, nb_filter=64, reduction=0.0, dropout_rate=0.3, weight_decay=1e-4, classes=1000, weights_path=None):
  compression = 1.0 - reduction
  nb_layers = [2,3,6,4] #[6,12,24,16]
  #input layer
  img_input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, CHANNELS), name='data')
  #initial convolution
  x = ZeroPadding2D((3,3), name='conv1_zeropadding')(img_input)
  x = Conv2D(nb_filter, (7,7), strides=(2,2), name = 'conv1', use_bias=False )(x)
  x = BatchNormalization(epsilon=eps, axis=concat_axis, name='conv1_bn')(x)
  x = Scale(axis= concat_axis, name='conv1_scale')(x)
  x = Activation(activation='relu', name= 'relu1')(x)
  x = ZeroPadding2D((1,1), name='pool1_zeropadding')(x)
  x = MaxPool2D((3,3), strides=(2,2), name= 'pool1')(x)

  #Adding dense blocks
  for i in range(nb_dense_block - 1):
    stage = i + 2
    x, nb_filter = dense_block(x, stage, nb_layers[i], nb_filter, growth_rate, dropout_rate=dropout_rate, weight_decay=weight_decay)

    #transition block
    x = transition_block(x, stage, nb_filter, compression=compression, dropout_rate=dropout_rate, weight_decay=weight_decay)
    nb_filter = int(nb_filter * compression)
  final_stage = stage + 1
  x, nb_filter = dense_block(x, final_stage, nb_layers[-1], nb_filter, growth_rate, dropout_rate=dropout_rate,  weight_decay=weight_decay)
  x = BatchNormalization(epsilon=eps, axis = concat_axis, name='conv'+str(final_stage)+'_blk_bn')(x)
  x = Scale(axis=concat_axis, name='conv'+str(final_stage)+'_blk_scale')(x)
  x = Activation('relu', name = 'relu' + str(final_stage) +'_blk' )(x)
  x = GlobalAveragePooling2D(name='pool' + str(final_stage) +'_blk')(x)
  x = Dense(classes, name='fc6')(x)
  x = Activation('softmax', name='prob')(x)
  model = Model(img_input, x, name='densenet')
  if weights_path is not None:
    model.load_weight(weights_path)
  model.summary()
  return model

In [1]:
DenseNetModel = DenseNet(classes=102)

NameError: name 'DenseNet' is not defined

In [ ]:
lr_fn = tf.optimizers.schedules.PiecewiseConstantDecay([100, 150, 200], [0.1, 0.05, 0.005, 0.0005])
DenseNetModel.compile(optimizer= tf.optimizers.Adam(0.0001), loss='categorical_crossentropy', 
                      metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

class mycallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy')>.95):
            print("\nReached 95% accuracy!")
            self.model.stop_training = True

myCallback = mycallback()

callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
             tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1), myCallback,
             tf.keras.callbacks.ModelCheckpoint('densenet_flower_class_mark03.h5', verbose=1, save_best_only=True, save_weights_only=True]

In [ ]:
'/input/training01/'+checkpoint_path

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
!kill 13961

In [ ]:
history = DenseNetModel.fit_generator(train_generator, steps_per_epoch=2*(train_generator.samples//train_generator.batch_size), validation_data=valid_generator,validation_steps=valid_generator.samples//valid_generator.batch_size , callbacks=callbacks, verbose=1, epochs=50, use_multiprocessing=True)

In [ ]:
DenseNetModel.load_weights('densenet_flower_class_mark03.h5')

In [ ]:
filenames = test_generator.filenames
nb_samples = len(filenames)

predict = DenseNetModel.predict_generator(test_generator,steps = nb_samples)

In [ ]:
DenseNetModel.evaluate_generator(test_generator,steps = nb_samples)